<a href="https://colab.research.google.com/github/Somabss1/AIML/blob/main/notebook/crewai_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index
!pip install llama-index-llms-groq
!pip install llama-index-core
!pip install llama-index-readers-file
!pip install llama-index-tools-wolfram-alpha
!pip install llama-index-embeddings-huggingface
!pip install 'crewai[tools]'
!pip install llama_index.llms.langchain

In [ ]:
from google.colab import userdata
from llama_index.llms.groq import Groq
groq_api_key = userdata.get('GROQ_API_KEY')
#
llm = Groq(model="llama3-70b-8192", api_key=groq_api_key)
#
response = llm.complete("Explain the importance of low latency LLMs")
print(response)



Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:

1. **Real-time Conversational AI**: In conversational AI, such as chatbots, voice assistants, and customer service platforms, low-latency LLMs enable rapid response times, creating a more natural and engaging user experience. Delays in response can lead to frustration and abandonment.
2. **Live Translation and Interpretation**: Low-latency LLMs facilitate real-time language translation and interpretation, enabling seamless communication across languages and cultures. This is particularly important in applications like conference calls, video conferencing, and live subtitles.
3. **Speech Recognition and Synthesis**: Fast and accurate speech recognition and synthesis are critical in applications like voice-controlled devices, voice assistants, and speech-to-text systems. Low-latency LLMs ensure t

In [ ]:
## Crewai requires a chat based model for binding
from langchain_openai import ChatOpenAI

chat_llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=groq_api_key,
    model="groq/llama3-70b-8192",
    temperature=0,
    max_tokens=1000,
)

In [ ]:
!wget "https://investors.endava.com/sec-filings/content/0001656081-24-000037/0001656081-24-000037.pdf" -O endava_20f.pdf

--2024-09-20 04:00:20--  https://investors.endava.com/sec-filings/content/0001656081-24-000037/0001656081-24-000037.pdf
Resolving investors.endava.com (investors.endava.com)... 3.101.49.108, 184.169.196.220
Connecting to investors.endava.com (investors.endava.com)|3.101.49.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘endava_20f.pdf’

endava_20f.pdf          [ <=>                ]   1.81M  10.9MB/s    in 0.2s    

2024-09-20 04:00:20 (10.9 MB/s) - ‘endava_20f.pdf’ saved [1894444]



In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI
import os
from langchain_openai import ChatOpenAI

#
reader = SimpleDirectoryReader(input_files=["endava_20f.pdf"])
docs = reader.load_data()
docs[1]

Document(id_='88bc3c3a-519f-4837-bed4-ce617a21fe4f', embedding=None, metadata={'page_label': '2', 'file_name': 'endava_20f.pdf', 'file_path': 'endava_20f.pdf', 'file_type': 'application/pdf', 'file_size': 1894444, 'creation_date': '2024-09-20', 'last_modified_date': '2024-09-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Indicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.\nx\t  Yes ¨\tNo\nIf this report is an annual or transition report, indicate by check mark if the registrant is not required to file reports pursuant to Section 13 or 15(d) of the \nSecurities Exchange Act of 1934.\n¨\t\t\t Yes x\tNo\nIndicate by check mark whether the registrant (1) has filed all reports required

In [ ]:
# Setup the embedding model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
index = VectorStoreIndex.from_documents(docs,
                                        embed_model=embed_model,
                                        )

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=5, llm=chat_llm)


In [ ]:
from crewai_tools import LlamaIndexTool
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Endava 2024 20F Query Tool",
    description="Use this tool to lookup the 2024 Endava Annual Report",
)
#
query_tool.args_schema.schema()

{'description': 'Schema for query tool.',
 'properties': {'query': {'description': 'Search query for the query tool.',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'QueryToolSchema',
 'type': 'object'}

In [ ]:
import os
from crewai import Agent, Task, Crew, Process

# Define your agents with roles and goals
researcher = Agent(
    role="Senior Financial Analyst",
    goal="Uncover insights about different tech companies",
    backstory="""You work at an asset management firm.
  Your goal is to understand  stocks like Endava.""",
    verbose=True,
    allow_delegation=False,
    tools=[query_tool],
    llm=chat_llm,
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    llm=chat_llm,
    verbose=True,
    allow_delegation=False,
)

In [ ]:
# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis of Endava's risk factors in 2024.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the headwinds that Uber faces.
  Your post should be informative yet accessible, catering to a casual audience.
  Make it sound cool, avoid complex words.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

In [ ]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=1,
)

In [ ]:
result = crew.kickoff()

print("######################")
print(result)


# Agent: Senior Financial Analyst
## Task: Conduct a comprehensive analysis of Endava's risk factors in 2024.

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/

AuthenticationError: litellm.AuthenticationError: AuthenticationError: GroqException - The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable